In [1]:
# Import Dependencies
import pandas as pd
import requests
import gmaps

In [2]:
# Import API key
from config import g_key

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [4]:
# 1. Read the WeatherPy_vacation.csv into a DataFrame.
vacation_df = pd.read_csv("../Vacation_Search/WeatherPy_vacation.csv")
vacation_df.head()

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,2,Terrasini,IT,84.22,clear sky,38.1460,13.0844,Florio Park Hotel
1,3,Baturaja,ID,75.20,overcast clouds,-4.1333,104.1667,"HOTEL Sekar Sunja, Baturaja Timur, Ogan Komeri..."
2,8,Bengkulu,ID,75.29,overcast clouds,-3.8004,102.2655,Grage Hotel Bengkulu
3,9,Bani,DO,86.86,few clouds,18.2833,-70.3333,Hospedaje Henriquez
4,15,Ribeira Grande,PT,75.60,broken clouds,38.5167,-28.7000,Quinta da Meia Eira


In [5]:
# 2. Using the template add the city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Current Description</dt><dd>{Current Description}
</dl>
"""

# 3a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in vacation_df.iterrows()]

# 3b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = vacation_df[["Lat", "Lng"]]
locations.head()

,Lat,Lng
0,38.1460,13.0844
1,-4.1333,104.1667
2,-3.8004,102.2655
3,18.2833,-70.3333
4,38.5167,-28.7000


In [6]:
# Add a heatmap of temperature for the vacation spots.
locations = vacation_df[["Lat", "Lng"]]
max_temp = vacation_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# 4a. Add a marker layer for each city to the map.
locations = vacation_df[["Lat", "Lng"]]
max_temp = vacation_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 4b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# From the map above pick 4 cities and create a vacation itinerary route to travel between the four cities. 
# 5. Create DataFrames for each city by filtering the 'vacation_df' using the loc method. 
# Hint: The starting and ending city should be the same city.

vacation_start_df = vacation_df.loc[vacation_df["City"] =='Miles City']
vacation_end_df = vacation_df.loc[vacation_df["City"] == 'Miles City']
vacation_stop1_df = vacation_df.loc[vacation_df["City"] == 'La Crosse']
vacation_stop2_df = vacation_df.loc[vacation_df["City"] == 'Jamestown'] 
vacation_stop3_df = vacation_df.loc[vacation_df["City"] == 'Athol'] 

vacation_start_df.head()

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
108,356,Miles City,US,83.37,clear sky,46.4083,-105.8406,FairBridge Inn & Suites Miles City


In [9]:
# 6. Get the latitude-longitude pairs as tuples from each city DataFrame using the to_numpy function and list indexing.

start = vacation_start_df["Lat"].to_numpy()[0], vacation_start_df["Lng"].to_numpy()[0]
end = vacation_end_df["Lat"].to_numpy()[0], vacation_end_df["Lng"].to_numpy()[0]
stop1 = vacation_stop1_df["Lat"].to_numpy()[0], vacation_stop1_df["Lng"].to_numpy()[0]
stop2 = vacation_stop2_df["Lat"].to_numpy()[0], vacation_stop2_df["Lng"].to_numpy()[0]
stop3 = vacation_stop3_df["Lat"].to_numpy()[0], vacation_stop3_df["Lng"].to_numpy()[0]

print(start)
print(end)
print(stop1)
print(stop2)
print(stop3)

(46.4083, -105.8406)
(46.4083, -105.8406)
(43.8014, -91.2396)
(42.097, -79.2353)
(42.5959, -72.2268)


In [10]:
# 7. Create a direction layer map using the start and end latitude-longitude pairs,
# and stop1, stop2, and stop3 as the waypoints. The travel_mode should be "DRIVING", "BICYCLING", or "WALKING".

fig = gmaps.figure()
vacation_itinerary = gmaps.directions_layer(
    start, end, waypoints=[stop1, stop2, stop3],
    travel_mode='DRIVING')

fig.add_layer(vacation_itinerary)

fig

Figure(layout=FigureLayout(height='420px'))

In [11]:
# 8. To create a marker layer map between the four cities.
#  Combine the four city DataFrames into one DataFrame using the concat() function.
itinerary_df = pd.concat([vacation_start_df, vacation_stop1_df, vacation_stop2_df, vacation_stop3_df, vacation_end_df],ignore_index=True)
itinerary_df

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,356,Miles City,US,83.37,clear sky,46.4083,-105.8406,FairBridge Inn & Suites Miles City
1,260,La Crosse,US,76.77,scattered clouds,43.8014,-91.2396,Holiday Inn & Suites La Crosse - Downtown
2,144,Jamestown,US,82.02,clear sky,42.0970,-79.2353,Hampton Inn & Suites Jamestown
3,492,Athol,US,88.48,few clouds,42.5959,-72.2268,Pequoig House Apartments
4,356,Miles City,US,83.37,clear sky,46.4083,-105.8406,FairBridge Inn & Suites Miles City


In [12]:
# 9 Using the template add city name, the country code, the weather description and maximum temperature for the city. 
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Current Description</dt><dd>{Current Description}
</dl>
"""

# 10a Get the data from each row and add it to the formatting template and store the data in a list.
vacation_info = [info_box_template.format(**row) for index, row in itinerary_df.iterrows()]
# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = itinerary_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map.
fig = gmaps.figure()
locations = itinerary_df[["Lat", "Lng"]]
vacation_itinerary = gmaps.directions_layer(
    start, end, waypoints=[stop1, stop2, stop3],
    travel_mode='DRIVING')
fig.add_layer(vacation_itinerary)
marker_layer = gmaps.marker_layer(locations, info_box_content=vacation_info)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))